In [1]:
# Imports

import os

import matplotlib.pyplot as plt
import numpy as np
import torch
import torchvision
from torchvision import transforms, models,datasets
from torch.utils.data import Dataset, DataLoader

In [2]:
# Download pretrained models on imagenet

resnet18 = models.resnet18(pretrained=True)
alexnet = models.alexnet(pretrained=True)
squeezenet = models.squeezenet1_0(pretrained=True)
vgg16 = models.vgg16(pretrained=True)
densenet = models.densenet161(pretrained=True)
inception = models.inception_v3(pretrained=True)
googlenet = models.googlenet(pretrained=True)
shufflenet = models.shufflenet_v2_x1_0(pretrained=True)
mobilenet_v2 = models.mobilenet_v2(pretrained=True)
mobilenet_v3_large = models.mobilenet_v3_large(pretrained=True)
mobilenet_v3_small = models.mobilenet_v3_small(pretrained=True)
resnext50_32x4d = models.resnext50_32x4d(pretrained=True)
wide_resnet50_2 = models.wide_resnet50_2(pretrained=True)
mnasnet = models.mnasnet1_0(pretrained=True)

In [3]:
# Simple Dog dataloader

batch_size = 500
transform = transforms.Compose(
    [transforms.Resize(256),
     transforms.CenterCrop(224),
     transforms.ToTensor(),
     transforms.Normalize(mean=[0.485, 0.456, 0.406],
                          std=[0.229, 0.224, 0.225])
])

class SimpleDog(Dataset):
    def __init__(self, image_path, transform=None):
        super(SimpleDog, self).__init__()
        self.data = datasets.ImageFolder(image_path, transform)
        
    def __getitem__(self, idx):
        x, y = self.data[idx]
        return x, y
    
    def __len__(self):
        return len(self.data)
    
    
img_path = r'./data/Stanford Dogs Dataset'
# print(os.listdir(img_path))

# dog_data = SimpleDog(r'./data/Stanford Dogs Dataset',  batch_size=1, transform=transform)

dog_data = datasets.ImageFolder(img_path, transform)
dog_loader = DataLoader(dog_data, batch_size=batch_size)
dog_iter = iter(dog_loader)

In [4]:
# Put data through multiple pretrained models
images, labels = dog_iter.next()

sm = torch.nn.Softmax(dim=1)

# Output from VGG1
output_vgg = vgg16(images)
output_vgg_simple = output_vgg[:, [196, 265]]
output_vgg_simple_sm = sm(output_vgg_simple)

# Output from Mobilenet
output_mobile = mobilenet_v3_large(images)
output_mobile_simple = output_mobile[:, [196, 265]]
output_mobile_simple_sm = sm(output_mobile_simple)

# Output from Googlenet
output_googlenet = googlenet(images)
output_googlenet_simple = output_googlenet[:, [196, 265]]
output_googlenet_simple_sm = sm(output_googlenet_simple)

C:\Users\lj_ji\anaconda3\envs\nn\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


In [5]:
def make_one_hot(labels, num_classes):
    one_hot = torch.zeros(len(labels), num_classes)
    for i, label in enumerate(labels):
        one_hot[i, label] = 1
    return one_hot


labels_one_hot = make_one_hot(labels, 2)

In [6]:
fusion_train_0 = torch.stack((output_vgg_simple_sm[:, 0], output_mobile_simple_sm[:, 0], output_googlenet_simple_sm[:, 0]), 1)
fusion_train_1 = torch.stack((output_vgg_simple_sm[:, 1], output_mobile_simple_sm[:, 1], output_googlenet_simple_sm[:, 1]), 1)

print(fusion_train_0.shape)
print(fusion_train_1.shape)

torch.Size([305, 3])
torch.Size([305, 3])


In [7]:
# ChINN

import torch
import numpy as np

# Convert decimal to binary string
def sources_and_subsets_nodes(N):
    str1 = "{0:{fill}"+str(N)+"b}"
    a = []
    for i in range(1,2**N):
        a.append(str1.format(i, fill='0'))

    sourcesInNode = []
    sourcesNotInNode = []
    subset = []
    sourceList = list(range(N))
    # find subset nodes of a node
    def node_subset(node, sourcesInNodes):
        return [node - 2**(i) for i in sourcesInNodes]
    
    # convert binary encoded string to integer list
    def string_to_integer_array(s, ch):
        N = len(s) 
        return [(N - i - 1) for i, ltr in enumerate(s) if ltr == ch]
    
    for j in range(len(a)):
        # index from right to left
        idxLR = string_to_integer_array(a[j],'1')
        sourcesInNode.append(idxLR)  
        sourcesNotInNode.append(list(set(sourceList) - set(idxLR)))
        subset.append(node_subset(j,idxLR))

    return sourcesInNode, subset


def subset_to_indices(indices):
    return [i for i in indices]

class Choquet_integral(torch.nn.Module):
    
    def __init__(self, N_in, N_out):
        super(Choquet_integral,self).__init__()
        self.N_in = N_in
        self.N_out = N_out
        self.nVars = 2**self.N_in - 2
        
        # The FM is initialized with mean
        dummy = (1./self.N_in) * torch.ones((self.nVars, self.N_out), requires_grad=True)
#        self.vars = torch.nn.Parameter( torch.Tensor(self.nVars,N_out))
        self.vars = torch.nn.Parameter(dummy)
        
        # following function uses numpy vs pytorch
        self.sourcesInNode, self.subset = sources_and_subsets_nodes(self.N_in)
        
        self.sourcesInNode = [torch.tensor(x) for x in self.sourcesInNode]
        self.subset = [torch.tensor(x) for x in self.subset]
        
    def forward(self,inputs):    
        self.FM = self.chi_nn_vars(self.vars)
        sortInputs, sortInd = torch.sort(inputs,1, True)
        M, N = inputs.size()
        sortInputs = torch.cat((sortInputs, torch.zeros(M,1)), 1)
        sortInputs = sortInputs[:,:-1] -  sortInputs[:,1:]
        
        out = torch.cumsum(torch.pow(2,sortInd),1) - torch.ones(1, dtype=torch.int64)
        
        data = torch.zeros((M,self.nVars+1))
        
        for i in range(M):
            data[i,out[i,:]] = sortInputs[i,:] 
        
        
        ChI = torch.matmul(data,self.FM)
            
        return ChI
    
    # Converts NN-vars to FM vars
    def chi_nn_vars(self, chi_vars):
#        nVars,_ = chi_vars.size()
        chi_vars = torch.abs(chi_vars)
        #        nInputs = inputs.get_shape().as_list()[1]
        
        FM = chi_vars[None, 0,:]
        for i in range(1,self.nVars):
            indices = subset_to_indices(self.subset[i])
            if (len(indices) == 1):
                FM = torch.cat((FM,chi_vars[None,i,:]),0)
            else:
                #         ss=tf.gather_nd(variables, [[1],[2]])
                maxVal,_ = torch.max(FM[indices,:],0)
                temp = torch.add(maxVal,chi_vars[i,:])
                FM = torch.cat((FM,temp[None,:]),0)
              
        FM = torch.cat([FM, torch.ones((1,self.N_out))],0)
        FM = torch.min(FM, torch.ones(1))  
        
        return FM

In [13]:
# Fusion 0
    
# training samples size
M = len(labels)

# number of inputs
N_in = 3

# number of outputs aka number of Choquet integral neurons
N_out = 1

# Create a synthetic dataset via random sampling from a normal distribution with mean =-1 and std=2
X_train = fusion_train_0

# Let's specify the FMs  (There will be N_out number of FMs)
# Herein we adopt binary encoding instead of lexicographic encoding to represent a FM that is easier to code. 
# As for example, an FM for three inputs using lexicographic encoding is, g = {g_1, g_2, g_3, g_{12}, g_{13}, g_{23}, g_{123}}.
# whereas its binary encoding is g = {g_1, g_2, g_{12}, g_3 g_{13}, g_{23}, g_{123}}.

# For simplicity, here we use OWA. 



# Generate the label or the groundtruth based on the provided FMs/OWAs. The labels are two dimentional
label_train = labels_one_hot[:, 0].unsqueeze_(-1)

# Now we want to recover the FMs from the training data and groundtruth
# First, build a Choquet integral neuron with N_in inputs and N_out outputs
net = Choquet_integral(N_in, N_out)

# set the optimization algorithms and paramters the learning
learning_rate = 0.3;

# Construct our loss function and an Optimizer. The call to model.parameters()
# in the SGD constructor will contain the learnable parameters of the two
# nn.Linear modules which are members of the model.
criterion = torch.nn.MSELoss(reduction='mean')
optimizer = torch.optim.SGD(net.parameters(), lr=learning_rate)   

num_epochs = 300;

# convert from numpy to torch tensor
X_train = torch.tensor(X_train,dtype=torch.float)
label_train = torch.tensor(label_train,dtype=torch.float)

# optimize
for t in range(num_epochs):
    # Forward pass: Compute predicted y by passing x to the model
    y_pred = net(X_train)

    # Compute the loss
    loss = criterion(y_pred, label_train)

    # Zero gradients, perform a backward pass, and update the weights.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()  

# Finally, the learned FMs
FM_learned = (net.chi_nn_vars(net.vars).cpu()).detach().numpy()
print('\n\nLearned FMs:')
print('FM1 = ', FM_learned[:,0])
# print('FM2 = ',FM_learned[:,1])





<ipython-input-13-a71bd0bfb1fb>:41: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train = torch.tensor(X_train,dtype=torch.float)
<ipython-input-13-a71bd0bfb1fb>:42: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  label_train = torch.tensor(label_train,dtype=torch.float)




Learned FMs:
FM1 =  [0.8525713  0.35048088 1.         0.12938859 1.         0.6285027
 1.        ]


In [14]:
# Fusion 1

# training samples size
M = len(labels)

# number of inputs
N_in = 3

# number of outputs aka number of Choquet integral neurons
N_out = 1

# Create a synthetic dataset via random sampling from a normal distribution with mean =-1 and std=2
X_train = fusion_train_1

# Let's specify the FMs  (There will be N_out number of FMs)
# Herein we adopt binary encoding instead of lexicographic encoding to represent a FM that is easier to code. 
# As for example, an FM for three inputs using lexicographic encoding is, g = {g_1, g_2, g_3, g_{12}, g_{13}, g_{23}, g_{123}}.
# whereas its binary encoding is g = {g_1, g_2, g_{12}, g_3 g_{13}, g_{23}, g_{123}}.

# For simplicity, here we use OWA. 



# Generate the label or the groundtruth based on the provided FMs/OWAs. The labels are two dimentional
label_train = labels_one_hot[:, 1].unsqueeze_(-1)

# Now we want to recover the FMs from the training data and groundtruth
# First, build a Choquet integral neuron with N_in inputs and N_out outputs
net = Choquet_integral(N_in, N_out)

# set the optimization algorithms and paramters the learning
learning_rate = 0.3;

# Construct our loss function and an Optimizer. The call to model.parameters()
# in the SGD constructor will contain the learnable parameters of the two
# nn.Linear modules which are members of the model.
criterion = torch.nn.MSELoss(reduction='mean')
optimizer = torch.optim.SGD(net.parameters(), lr=learning_rate)   

num_epochs = 300;

# convert from numpy to torch tensor
X_train = torch.tensor(X_train,dtype=torch.float)
label_train = torch.tensor(label_train,dtype=torch.float)

# optimize
for t in range(num_epochs):
    # Forward pass: Compute predicted y by passing x to the model
    y_pred = net(X_train)

    # Compute the loss
    loss = criterion(y_pred, label_train)

    # Zero gradients, perform a backward pass, and update the weights.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()  

# Finally, the learned FMs
FM_learned = (net.chi_nn_vars(net.vars).cpu()).detach().numpy()
print('\n\nLearned FMs:')
print('FM1 = ', FM_learned[:,0])
# print('FM2 = ',FM_learned[:,1])


<ipython-input-14-3e469623b98e>:41: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train = torch.tensor(X_train,dtype=torch.float)
<ipython-input-14-3e469623b98e>:42: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  label_train = torch.tensor(label_train,dtype=torch.float)




Learned FMs:
FM1 =  [0.4537986  0.01906862 0.9568969  0.01494275 0.6896283  0.0256345
 1.        ]


NameError: name 'fusion_train_2' is not defined